In [ ]:
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
dataset_dir_train = '/content/drive/My Drive/Penoumia/train'
dataset_dir_test = '/content/drive/My Drive/Penoumia/test'
dataset_dir_val = '/content/drive/My Drive/Penoumia/val'
os.listdir(dataset_dir_train)
os.listdir(dataset_dir_test)
os.listdir(dataset_dir_val)

['NORMAL', 'PNEUMONIA']

In [ ]:
normal_image_train = []
for im_path in glob.glob(dataset_dir_train + '/NORMAL/*'):
  normal_image_train.append(mpimg.imread(im_path))
print (len(normal_image_train))

In [ ]:
covid_image_train = []
for im_path in glob.glob(dataset_dir_train + '/PNEUMONIA/*'):
  covid_image_train.append(mpimg.imread(im_path))
print(len(covid_image_train))

In [ ]:
normal_image_test = []
for im_path in glob.glob(dataset_dir_test + '/NORMAL/*'):
  normal_image_test.append(mpimg.imread(im_path))
print (len(normal_image_test))
covid_image_test = []
for im_path in glob.glob(dataset_dir_test+ '/PNEUMONIA/*'):
  covid_image_test.append(mpimg.imread(im_path))
print(len(covid_image_test))

In [ ]:
fig_norm = plt.figure()
fig_norm.suptitle('Normal')
plt.imshow(normal_image_train[0], cmap='gray')

In [ ]:
fig_cov = plt.figure()
fig_cov.suptitle('Covid')
plt.imshow(covid_image_train[0], cmap='gray')

In [ ]:
img_he = 150
img_wi = 150
n_channels = 3

INPUT_SHAPE = (img_he, img_wi, n_channels)
num_classes = 2  # Binary classification
no_epoches = 150
batch_size = 50

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), input_shape =INPUT_SHAPE))
model.add(Activation ('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3,3)))
model.add(Activation ('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Conv2D(250, (3,3)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.5))

model.add(Conv2D(256, (2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(32))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, 
                                   zoom_range=0.2, horizontal_flip=True, validation_split=0.3)

train_generator = train_datagen.flow_from_directory(dataset_dir_train, target_size=(img_he, img_wi), 
                                                    batch_size=batch_size, class_mode='binary', subset='training')

In [ ]:
validation_generator = train_datagen.flow_from_directory(dataset_dir_test, target_size=(img_he, img_wi),
                                                         batch_size=batch_size, class_mode='binary'
                                                         , shuffle = False, subset='validation')

In [ ]:
history = model.fit(train_generator, steps_per_epoch = train_generator.samples//batch_size, 
                              validation_data = validation_generator,
                              validation_steps = validation_generator.samples //batch_size, epochs = no_epoches)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from keras import applications
from keras import optimizers
# ResNet50 with pre-trained weights
base_model = applications.ResNet50(weights='imagenet', include_top=False)
train_after_layer = 25  
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
# Add your top layer block
model2 = Model(base_model.input, predictions)

# set the first K layers (up to the last conv block)
# to non-trainable (weights will not change during training)
for layer in model2.layers[:train_after_layer]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model2.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, 
                                   zoom_range=0.2, horizontal_flip=True, validation_split=0.3)

train_generator = train_datagen.flow_from_directory(dataset_dir, target_size=(img_he, img_wi), 
                                                    batch_size=batch_size, class_mode='binary', subset='training')
validation_generator = train_datagen.flow_from_directory(dataset_dir, target_size=(img_he, img_wi),
                                                         batch_size=batch_size, class_mode='binary'
                                                         , shuffle = False, subset='validation')
epochs_fine = 100

model2.fit_generator(train_generator, steps_per_epoch = train_generator.samples//batch_size, 
                              validation_data = validation_generator,
                              validation_steps = validation_generator.samples //batch_size, epochs = epochs_fine)

model2.save_weights('finetuned.h5')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()